### Import Dependencies

In [ ]:
# Pygame intended for later use
!pip install pygame

In [ ]:
# Gym Imports
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete # different types of spaces

# Helpers
import numpy as np
import random
import os

# Stable baselines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Time module to make program halt for presentation purposes
import time

In [ ]:
# Pygame
import pygame as pg


screen_width = 800
screen_height = 600

pg.init()
screen = pg.display.set_mode((screen_width,screen_height))
pg.display.set_caption("Shower")

# see if we can load more than standard BMP
if not pg.image.get_extended():
    raise SystemExit("Sorry, extended image module required")

# game constants
SCREENRECT = pg.Rect(0, 0, screen_width, screen_height)

pg.quit()

In [ ]:
def load_image(file):
    """loads an image, prepares it for play"""
    file = os.path.join(file)
    try:
        surface = pg.image.load(file)
    except pg.error:
        raise SystemExit('Could not load image "%s" %s' % (file, pg.get_error()))
    return surface.convert()

In [ ]:
pg.init()
screen = pg.display.set_mode((screen_width,screen_height))
pg.display.set_caption("Shower")

class Knob(pg.sprite.Sprite):
    """A visual for what way the AI is turning the temperature"""
    images = [load_image("assets/sprites/knob_none.png"), 
              load_image("assets/sprites/knob_r.png"), 
              load_image("assets/sprites/knob_l.png")]

    def __init__(self, pos):
        pg.sprite.Sprite.__init__(self, self.containers)
        self.image = self.images[0]
        self.rect = self.image.get_rect()
        self.rect.x = pos[0] - (self.rect.width / 2.0)
        self.rect.y = pos[1] - (self.rect.height / 2.0)

    def update(self, action: int):
        self.image = self.images[action]

pg.display.quit()

In [ ]:
# Initialize Game Groups
all = pg.sprite.RenderUpdates()
    
Knob.containers = all

### Building an Environment
- Build an agent to give us best shower possible
- Randomly temperature
- 37 and 39 degrees range

In [ ]:
pg.display.init()
class ShowerEnv(Env):
    visual = Knob((400,300))
    GAME_FONT = pg.freetype.Font("assets/fonts/arial.ttf", 24)
    
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60

    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1
        self.visual.update(action)

        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 

        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False

        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        pg.time.wait(60)
          
        # Return step information
        return self.state, reward, done, info
    
    def drawText(self, screen, string, pos):
        self.GAME_FONT.render_to(screen, pos, string, (250, 250, 250))

    def render(self):
        # create the background, tile the bgd image
        bgdtile = load_image("assets/sprites/background.png")
        background = pg.Surface(SCREENRECT.size)
        for x in range(0, SCREENRECT.width, bgdtile.get_width()):
            background.blit(bgdtile, (x, 0))
        screen.blit(background, (0, 0))
        
         # draw the scene
        all.clear(screen, background) # clear/erase the last drawn sprites
        self.drawText(screen, "Temperature: " + str(self.state), (300, 200))
        dirty = all.draw(screen)
        pg.display.update(dirty)
        pg.display.flip()
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60
        return self.state
pg.display.quit()

### Test Environment

In [ ]:
env = ShowerEnv()

In [ ]:
# Run this cell to test that the Environment works properly
# This will randomly pick from a Discrete action step, no model is used here.

pg.display.init()
screen = pg.display.set_mode((screen_width,screen_height))
pg.display.set_caption("Shower")

episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        pg.event.get()
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward         
    print('Episode:{} Score:{}'.format(episode, score))
env.close()
pg.display.quit()

### Train Model

In [ ]:
log_path = os.path.join('training', 'logs')
#model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_13')

In [ ]:
!tensorboard --logdir={training_log_path}

### Save Model

In [ ]:
shower_path = os.path.join('training', 'saved_models', 'ShowerModel_PPO')

In [ ]:
model = PPO.load(shower_path, env)

In [ ]:
pg.display.init()
screen = pg.display.set_mode((screen_width,screen_height))
pg.display.set_caption("Shower")

episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        pg.event.get()
        env.render()
        action, _ = model.predict(obs) # predict returns two, but we only require action
        obs, reward, done, info = env.step(action)
        score += reward         
    print('Episode:{} Score:{}'.format(episode, score))
env.close()
pg.display.quit()